In [1]:
%reset -fs
import bs4
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import requests
import pickle
import re

In [2]:
df = pd.read_pickle('df_all_movies_2019_pk')
df

,Rank,Movie,ReleaseDate,Distributor,Genre,Revenues,Tickets_Sold,summary_page_url,cast_page_url
0,1,Avengers: Endgame,"Apr 26, 2019",Walt Disney,Action,"$858,373,000","93,708,843",https://www.the-numbers.com/movie/Avengers-End...,https://www.the-numbers.com/movie/Avengers-End...
1,2,The Lion King,"Jul 19, 2019",Walt Disney,Adventure,"$543,638,043","59,349,131",https://www.the-numbers.com/movie/Lion-King-Th...,https://www.the-numbers.com/movie/Lion-King-Th...
2,3,Frozen II,"Nov 22, 2019",Walt Disney,Adventure,"$450,439,533","49,174,621",https://www.the-numbers.com/movie/Frozen-II-(2...,https://www.the-numbers.com/movie/Frozen-II-(2...
3,4,Toy Story 4,"Jun 21, 2019",Walt Disney,Adventure,"$434,038,008","47,384,062",https://www.the-numbers.com/movie/Toy-Story-4-...,https://www.the-numbers.com/movie/Toy-Story-4-...
4,5,Captain Marvel,"Mar 8, 2019",Walt Disney,Action,"$426,829,839","46,597,144",https://www.the-numbers.com/movie/Captain-Marv...,https://www.the-numbers.com/movie/Captain-Marv...
...,...,...,...,...,...,...,...,...,...
675,676,Chef Flynn,"Nov 9, 2018",Kino Lorber,Documentary,$721,78,https://www.the-numbers.com/movie/Chef-Flynn-(...,https://www.the-numbers.com/movie/Chef-Flynn-(...
676,677,I am Not a Witch,"Sep 7, 2018",Film Movement,Drama,$550,60,https://www.the-numbers.com/movie/I-am-Not-a-W...,https://www.the-numbers.com/movie/I-am-Not-a-W...
677,678,Divide and Conquer: The Sto…,"Dec 7, 2018",Magnolia Pictures,Documentary,$474,51,https://www.the-numbers.com/movie/Divide-and-C...,https://www.the-numbers.com/movie/Divide-and-C...
678,679,Senso,"Jul 8, 1968",Fleetwood Films,Drama,$437,47,https://www.the-numbers.com/movie/Senso-(Italy...,https://www.the-numbers.com/movie/Senso-(Italy...


In [3]:
row_count = df.shape[0]
row_count

680

In [4]:

r = requests.get(df.loc[i,'summary_page_url']) #location in my dataframe
s = bs(r.text)

ex_all_tables = s.find_all('table')
ex_row = ex_all_tables[2].find_all('tr')
ex_row

[<tr><td><b>Opening Weekend:</b></td><td>$357,115,007 (41.6% of total gross)</td></tr>,
 <tr><td><b>Legs:</b></td><td>2.40 (domestic box office/biggest weekend)</td></tr>,
 <tr><td><b>Domestic Share:</b></td><td>30.7% (domestic box office/worldwide)</td></tr>,
 <tr><td><b>Production Budget:</b></td><td>$400,000,000 (worldwide box office is 7.0 times production budget)</td></tr>,
 <tr><td><b>Theater counts:</b></td><td>4,662 opening theaters/4,662 max. theaters, 8.1 weeks average run per theater</td></tr>,
 <tr>
 <td><b>Infl. Adj. Dom. BO</b></td>
 <td>$858,373,002</td>
 </tr>]

In [5]:
# Extract legs trial
ex_table_metrics = ex_all_tables[2]
all_rows_metrics = ex_table_metrics.find_all('tr')
for row in all_rows_metrics:
    found_legs = False
    if ('Legs' in row.text):
        ex_rows_legs = row
        found_legs = True
        break

if (found_legs):
    ex_strg_legs = ex_rows_legs.text
    print(ex_strg_legs)

    pattern = re.compile(r"[-+]?\d*\.\d+|\d+")
    print(pattern.findall(ex_strg_legs))
    legs = float(pattern.findall(ex_strg_legs)[0])
else:
    legs = 'not_found'
        
legs

Legs:2.40 (domestic box office/biggest weekend)
['2.40']


2.4

In [6]:
# Extract production budget trial

ex_row_budget = ex_table_metrics.find_all('tr')[3]
ex_strg_budget = ex_row_budget.text
print(ex_strg_budget)

pattern = re.compile(r"[-+]?(?:\d*\.\d+|\d+)")

print(pattern.findall(ex_strg_budget))
budget_ratio_list = pattern.findall(ex_strg_budget)


ratio = float(budget_ratio_list.pop())
print(ratio)

budget_str = ''
budget_str = budget_str.join(budget_ratio_list)
print(budget_str)

budget = int(budget_str)
budget

Production Budget:$400,000,000 (worldwide box office is 7.0 times production budget)
['400', '000', '000', '7.0']
7.0
400000000


400000000

In [7]:
#extract theater count trial

ex_row_theater_count = ex_table_metrics.find_all('tr')[4]
ex_strg_theater_count = ex_row_theater_count.find_all('td')[1].text

pattern = re.compile(r"[-+]?(?:\d*\.\d+|\d+)")
print(pattern.findall(ex_strg_theater_count))
theater_count_list = pattern.findall(ex_strg_theater_count)

theater_count = int(theater_count_list[0] + theater_count_list[1])
print(theater_count)

theater_run = float(theater_count_list[-1])
print(theater_run)

['4', '662', '4', '662', '8.1']
4662
8.1


In [8]:
#Extract Movie Details table
ex_table_movie_details = ex_all_tables[5]

In [9]:
#Extract franchise trial

ex_row_franchise = ex_table_movie_details.find_all('tr')[5]
ex_elems = ex_row_franchise.find_all('td')[1].find_all('a')
for i in ex_elems:
    print(i.text)

Marvel Cinematic Universe
Avengers


In [10]:
#Extract source trial
ex_row_source = ex_table_movie_details.find_all('tr')[8]
source = ex_row_source.find_all('td')[1].find_all('a')[0].text
source

'Based on Comic/Graphic Novel'

In [11]:
#Extract Genre trial
ex_row_genre = ex_table_movie_details.find_all('tr')[9]
genre = ex_row_genre.find_all('td')[1].find_all('a')[0].text
genre

'Action'

In [12]:
# Extract creative type trial
ex_row_ctype = ex_table_movie_details.find_all('tr')[11]
ctype = ex_row_ctype.find_all('td')[1].find_all('a')[0].text
ctype

'Super Hero'

In [13]:
# Extract legs for all 2019 movies  v2

# all_movies_legs_list = []
# all_movies_all_tables_2019_list = []

# for i in range(row_count):
#     r = requests.get(df.loc[i,'summary_page_url']) #location in my dataframe
#     s = bs(r.text)
#     ex_all_tables = s.find_all('table')
# #     all_movies_all_tables_2019_list.append(ex_all_tables)
    
#     for table in ex_all_tables:
#         all_rows = table.find_all('tr')
#         found_legs = False
    
#         for row in all_rows:
#             if ('Legs' in row.text):
#                 ex_rows_legs = row
#                 found_legs = True

#                 ex_strg_legs = ex_rows_legs.text
#                 print(ex_strg_legs)

#                 pattern = re.compile(r"[-+]?\d*\.\d+|\d+")
#                 print(pattern.findall(ex_strg_legs))
#                 legs = float(pattern.findall(ex_strg_legs)[0])
#                 all_movies_legs_list.append(legs)
#                 break        
#         if (found_legs):
#             break
                
#     if (not found_legs):
#         legs = 1
#         all_movies_legs_list.append(legs)
    
#     print(legs)
    

# legs_2019_df = pd.DataFrame(all_movies_legs_list, columns = ['Legs'])
# legs_2019_df.to_pickle('legs_2019_df_pk')



In [14]:
# legs_2019_df

In [24]:
# Extract budget and ratio for all 2019 movies v2

# all_movies_budget_list = []
# all_movies_wbo_to_budget_ratio_list = []


# for i in range(row_count):
#     r = requests.get(df.loc[i,'summary_page_url']) #location in my dataframe
#     s = bs(r.text)
#     ex_all_tables = s.find_all('table')
    
#     for table in ex_all_tables:
#         all_rows = table.find_all('tr')
#         found_budget = False
    
#         for row in all_rows:
#             if ('Budget' in row.text):
#     #             print(row.text)
#                 found_budget = True
#                 ex_row_budget = row
#                 ex_strg_budget = ex_row_budget.text
#     #             print(ex_strg_budget)

#                 pattern = re.compile(r"[-+]?(?:\d*\.\d+|\d+)")

#     #             print(pattern.findall(ex_strg_budget))
#                 budget_ratio_list = pattern.findall(ex_strg_budget)


#                 ratio = float(budget_ratio_list.pop())
#                 print(ratio)

#                 budget_str = ''
#                 budget_str = budget_str.join(budget_ratio_list)
#                 print(budget_str)

#                 budget = int(budget_str)

#                 all_movies_budget_list.append(budget)
#                 all_movies_wbo_to_budget_ratio_list.append(ratio)
#                 break
#         if(found_budget):
#             break

#     if (not found_budget):
#         budget = 'nf'
#         ratio = 'nf'
#         all_movies_budget_list.append(budget)
#         all_movies_wbo_to_budget_ratio_list.append(ratio)
    
    
#     print(all_movies_budget_list)
#     print(all_movies_wbo_to_budget_ratio_list)
    
    
# budget_2019_df = pd.DataFrame(all_movies_budget_list, 
#                                   columns = ['Production_Budget'])
# wbo_to_budget_ratio_2019_df = pd.DataFrame(all_movies_wbo_to_budget_ratio_list,
#                                                columns = ['WBO to Budget Ratio'])
     
# budget_2019_df.to_pickle('budget_2019_pk')
# wbo_to_budget_ratio_2019_df.to_pickle('wbo_to_budget_ratio_2019_pk')

In [25]:
#  budget_2019_df

In [76]:
# Extract theater count and average run for all 2019 movies

# all_movies_theater_count_list = []
# all_movies_avg_run_list = []

# for i in range(row_count):
#     r = requests.get(df.loc[i,'summary_page_url']) #location in my dataframe
#     s = bs(r.text)
#     ex_all_tables = s.find_all('table')
#     count = i

#     for table in ex_all_tables:
#         all_rows = table.find_all('tr')
#         found_theater = False
        
#         for row in all_rows:
#             if ('Theater' in row.text and 'counts' in row.text):
#                 found_theater = True

#                 ex_row_theater_count = row
#                 print(row.text)
#                 ex_strg_theater_count = row.text

#                 pattern1 = re.compile(r"[-+]?(?:\d*\.\d+|\d+)")
#                 pattern2 = re.compile(r"[-+]?(?:\d*\,\d+|\d+)")
#                 print(pattern1.findall(ex_strg_theater_count))
#                 print(pattern2.findall(ex_strg_theater_count))
                
#                 theater_count_list1 = pattern1.findall(ex_strg_theater_count)
#                 theater_count_list2 = pattern2.findall(ex_strg_theater_count)
 
#                 theater_run = float(theater_count_list1[-1])
#                 print(theater_run)
#                 theater_count = int(theater_count_list2[0].replace(',', ''))
#                 print(theater_run)

#                 all_movies_theater_count_list.append(theater_count)
#                 all_movies_avg_run_list.append(theater_run)
#                 break
           
#         if(found_theater):
#             break     
                
#     if(not found_theater):
#         theater_count = 'nf'
#         theater_run = 'nf'

#         all_movies_theater_count_list.append(theater_count)
#         all_movies_avg_run_list.append(theater_run)
            
            
# theater_count_2019_df = pd.DataFrame(all_movies_theater_count_list,
#                                      columns = ['Theater_Count'])
# avg_run_2019_df = pd.DataFrame(all_movies_avg_run_list,
#                                columns = ['Average_Run'])
       

# theater_count_2019_df.to_pickle('theater_count_2019_pk')
# avg_run_2019_df.to_pickle('avg_run_2019_pk')

Theater counts:4,662 opening theaters/4,662 max. theaters, 8.1 weeks average run per theater
['4', '662', '4', '662', '8.1']
['4,662', '4,662', '8', '1']
8.1
8.1
Theater counts:4,725 opening theaters/4,802 max. theaters, 8.5 weeks average run per theater
['4', '725', '4', '802', '8.5']
['4,725', '4,802', '8', '5']
8.5
8.5
Theater counts:4,440 opening theaters/4,440 max. theaters, 8.6 weeks average run per theater
['4', '440', '4', '440', '8.6']
['4,440', '4,440', '8', '6']
8.6
8.6
Theater counts:4,575 opening theaters/4,575 max. theaters, 8.3 weeks average run per theater
['4', '575', '4', '575', '8.3']
['4,575', '4,575', '8', '3']
8.3
8.3
Theater counts:4,310 opening theaters/4,310 max. theaters, 7.9 weeks average run per theater
['4', '310', '4', '310', '7.9']
['4,310', '4,310', '7', '9']
7.9
7.9
Theater counts:4,406 opening theaters/4,406 max. theaters, 6.7 weeks average run per theater
['4', '406', '4', '406', '6.7']
['4,406', '4,406', '6', '7']
6.7
6.7
Theater counts:4,634 opening

Theater counts:3,585 opening theaters/3,585 max. theaters, 3.7 weeks average run per theater
['3', '585', '3', '585', '3.7']
['3,585', '3,585', '3', '7']
3.7
3.7
Theater counts:2,912 opening theaters/2,912 max. theaters, 4.4 weeks average run per theater
['2', '912', '2', '912', '4.4']
['2,912', '2,912', '4', '4']
4.4
4.4
Theater counts:25 opening theaters/2,648 max. theaters, 9.8 weeks average run per theater
['25', '2', '648', '9.8']
['25', '2,648', '9', '8']
9.8
9.8
Theater counts:3,460 opening theaters/3,460 max. theaters, 3.8 weeks average run per theater
['3', '460', '3', '460', '3.8']
['3,460', '3,460', '3', '8']
3.8
3.8
Theater counts:3,444 opening theaters/3,444 max. theaters, 4.6 weeks average run per theater
['3', '444', '3', '444', '4.6']
['3,444', '3,444', '4', '6']
4.6
4.6
Theater counts:3,642 opening theaters/3,642 max. theaters, 3.6 weeks average run per theater
['3', '642', '3', '642', '3.6']
['3,642', '3,642', '3', '6']
3.6
3.6
Theater counts:3,502 opening theaters/3,

Theater counts:17 opening theaters/1,490 max. theaters, 5.3 weeks average run per theater
['17', '1', '490', '5.3']
['17', '1,490', '5', '3']
5.3
5.3
Theater counts:3,652 opening theaters/3,652 max. theaters, 3.1 weeks average run per theater
['3', '652', '3', '652', '3.1']
['3,652', '3,652', '3', '1']
3.1
3.1
Theater counts:3,380 opening theaters/3,380 max. theaters, 3.2 weeks average run per theater
['3', '380', '3', '380', '3.2']
['3,380', '3,380', '3', '2']
3.2
3.2
Theater counts:1,059 opening theaters/1,516 max. theaters, 3.6 weeks average run per theater
['1', '059', '1', '516', '3.6']
['1,059', '1,516', '3', '6']
3.6
3.6
Theater counts:4 opening theaters/1,554 max. theaters, 6.2 weeks average run per theater
['4', '1', '554', '6.2']
['4', '1,554', '6', '2']
6.2
6.2
Theater counts:2,442 opening theaters/2,534 max. theaters, 5.3 weeks average run per theater
['2', '442', '2', '534', '5.3']
['2,442', '2,534', '5', '3']
5.3
5.3
Theater counts:2,502 opening theaters/2,502 max. theate

Theater counts:922 opening theaters/922 max. theaters, 2.0 weeks average run per theater
['922', '922', '2.0']
['922', '922', '2', '0']
2.0
2.0
Theater counts:2,073 opening theaters/2,073 max. theaters, 2.0 weeks average run per theater
['2', '073', '2', '073', '2.0']
['2,073', '2,073', '2', '0']
2.0
2.0
Theater counts:873 opening theaters/873 max. theaters, 1.4 weeks average run per theater
['873', '873', '1.4']
['873', '873', '1', '4']
1.4
1.4
Theater counts:48 opening theaters/288 max. theaters, 4.9 weeks average run per theater
['48', '288', '4.9']
['48', '288', '4', '9']
4.9
4.9
Theater counts:1,495 opening theaters/1,501 max. theaters, 2.2 weeks average run per theater
['1', '495', '1', '501', '2.2']
['1,495', '1,501', '2', '2']
2.2
2.2
Theater counts:7 opening theaters/207 max. theaters, 5.2 weeks average run per theater
['7', '207', '5.2']
['7', '207', '5', '2']
5.2
5.2
Theater counts:3 opening theaters/263 max. theaters, 6.3 weeks average run per theater
['3', '263', '6.3']
['

Theater counts:5 opening theaters/428 max. theaters, 2.3 weeks average run per theater
['5', '428', '2.3']
['5', '428', '2', '3']
2.3
2.3
Theater counts:268 opening theaters/268 max. theaters, 3.2 weeks average run per theater
['268', '268', '3.2']
['268', '268', '3', '2']
3.2
3.2
Theater counts:3 opening theaters/136 max. theaters, 5.0 weeks average run per theater
['3', '136', '5.0']
['3', '136', '5', '0']
5.0
5.0
Theater counts:4 opening theaters/541 max. theaters, 7.4 weeks average run per theater
['4', '541', '7.4']
['4', '541', '7', '4']
7.4
7.4
Theater counts:1,744 opening theaters/1,950 max. theaters, 6.4 weeks average run per theater
['1', '744', '1', '950', '6.4']
['1,744', '1,950', '6', '4']
6.4
6.4
Theater counts:4,103 opening theaters/4,148 max. theaters, 7.4 weeks average run per theater
['4', '103', '4', '148', '7.4']
['4,103', '4,148', '7', '4']
7.4
7.4
Theater counts:11 opening theaters/3,987 max. theaters, 7.2 weeks average run per theater
['11', '3', '987', '7.2']
['

...

Theater counts:23 opening theaters/23 max. theaters, 1.6 weeks average run per theater
['23', '23', '1.6']
['23', '23', '1', '6']
1.6
1.6
Theater counts:4 opening theaters/25 max. theaters, 5.4 weeks average run per theater
['4', '25', '5.4']
['4', '25', '5', '4']
5.4
5.4
Theater counts:2 opening theaters/27 max. theaters, 8.2 weeks average run per theater
['2', '27', '8.2']
['2', '27', '8', '2']
8.2
8.2
Theater counts:3 opening theaters/8 max. theaters, 1.0 weeks average run per theater
['3', '8', '1.0']
['3', '8', '1', '0']
1.0
1.0
Theater counts:2 opening theaters/9 max. theaters, 6.3 weeks average run per theater
['2', '9', '6.3']
['2', '9', '6', '3']
6.3
6.3
Theater counts:22 opening theaters/22 max. theaters, 1.7 weeks average run per theater
['22', '22', '1.7']
['22', '22', '1', '7']
1.7
1.7
Theater counts:2 opening theaters/7 max. theaters, 3.3 weeks average run per theater
['2', '7', '3.3']
['2', '7', '3', '3']
3.3
3.3
Theater counts:3 opening theaters/12 max. theaters, 3.5 we

Theater counts:1 opening theaters/10 max. theaters, 0.8 weeks average run per theater
['1', '10', '0.8']
['1', '10', '0', '8']
0.8
0.8
Theater counts:15 opening theaters/15 max. theaters, 1.5 weeks average run per theater
['15', '15', '1.5']
['15', '15', '1', '5']
1.5
1.5
Theater counts:2 opening theaters/2 max. theaters, 1.0 weeks average run per theater
['2', '2', '1.0']
['2', '2', '1', '0']
1.0
1.0
Theater counts:5 opening theaters/6 max. theaters, 3.3 weeks average run per theater
['5', '6', '3.3']
['5', '6', '3', '3']
3.3
3.3
Theater counts:1 opening theaters/7 max. theaters, 1.0 weeks average run per theater
['1', '7', '1.0']
['1', '7', '1', '0']
1.0
1.0
Theater counts:2 opening theaters/10 max. theaters, 2.9 weeks average run per theater
['2', '10', '2.9']
['2', '10', '2', '9']
2.9
2.9
Theater counts:1 opening theaters/6 max. theaters, 2.8 weeks average run per theater
['1', '6', '2.8']
['1', '6', '2', '8']
2.8
2.8
Theater counts:3 opening theaters/16 max. theaters, 4.8 weeks av

Theater counts:10 opening theaters/10 max. theaters, 1.0 weeks average run per theater
['10', '10', '1.0']
['10', '10', '1', '0']
1.0
1.0
Theater counts:1 opening theaters/3 max. theaters, 5.7 weeks average run per theater
['1', '3', '5.7']
['1', '3', '5', '7']
5.7
5.7
Theater counts:24 opening theaters/24 max. theaters
['24', '24']
['24', '24']
24.0
24.0
Theater counts:1 opening theaters/3 max. theaters, 2.7 weeks average run per theater
['1', '3', '2.7']
['1', '3', '2', '7']
2.7
2.7
Theater counts:3 opening theaters/5 max. theaters, 1.8 weeks average run per theater
['3', '5', '1.8']
['3', '5', '1', '8']
1.8
1.8
Theater counts:1 opening theaters/3 max. theaters, 5.0 weeks average run per theater
['1', '3', '5.0']
['1', '3', '5', '0']
5.0
5.0
Theater counts:2 opening theaters/31 max. theaters, 1.3 weeks average run per theater
['2', '31', '1.3']
['2', '31', '1', '3']
1.3
1.3
Theater counts:32 opening theaters/32 max. theaters, 1.0 weeks average run per theater
['32', '32', '1.0']
['32

Theater counts:1 opening theaters/1 max. theaters, 2.0 weeks average run per theater
['1', '1', '2.0']
['1', '1', '2', '0']
2.0
2.0
Theater counts:1 opening theaters/2 max. theaters, 2.5 weeks average run per theater
['1', '2', '2.5']
['1', '2', '2', '5']
2.5
2.5
Theater counts:5 opening theaters/5 max. theaters, 3.0 weeks average run per theater
['5', '5', '3.0']
['5', '5', '3', '0']
3.0
3.0
Theater counts:10 opening theaters/10 max. theaters, 1.0 weeks average run per theater
['10', '10', '1.0']
['10', '10', '1', '0']
1.0
1.0
Theater counts:1 opening theaters/4 max. theaters, 5.3 weeks average run per theater
['1', '4', '5.3']
['1', '4', '5', '3']
5.3
5.3
Theater counts:1 opening theaters/1 max. theaters, 2.0 weeks average run per theater
['1', '1', '2.0']
['1', '1', '2', '0']
2.0
2.0
Theater counts:2 opening theaters/2 max. theaters, 3.5 weeks average run per theater
['2', '2', '3.5']
['2', '2', '3', '5']
3.5
3.5
Theater counts:1 opening theaters/1 max. theaters, 1.0 weeks average r

Theater counts:3 opening theaters/3 max. theaters, 1.0 weeks average run per theater
['3', '3', '1.0']
['3', '3', '1', '0']
1.0
1.0
Theater counts:1 opening theaters/4 max. theaters, 3.8 weeks average run per theater
['1', '4', '3.8']
['1', '4', '3', '8']
3.8
3.8
Theater counts:1 opening theaters/10 max. theaters, 3.4 weeks average run per theater
['1', '10', '3.4']
['1', '10', '3', '4']
3.4
3.4
Theater counts:2 opening theaters/8 max. theaters, 6.0 weeks average run per theater
['2', '8', '6.0']
['2', '8', '6', '0']
6.0
6.0
Theater counts:15 opening theaters/16 max. theaters, 1.5 weeks average run per theater
['15', '16', '1.5']
['15', '16', '1', '5']
1.5
1.5
Theater counts:4 opening theaters/377 max. theaters, 0.0 weeks average run per theater
['4', '377', '0.0']
['4', '377', '0', '0']
0.0
0.0


In [77]:
# len(all_movies_theater_count_list)
# avg_run_2019_df

,Average_Run
0,8.1
1,8.5
2,8.6
3,8.3
4,7.9
...,...
675,3.4
676,6.0
677,1.5
678,nf


In [78]:
# theater_count_2019_df

,Theater_Count
0,4662
1,4725
2,4440
3,4575
4,4310
...,...
675,1
676,2
677,15
678,nf


In [80]:
# all_movies_source_list = []

# for i in range(row_count):
#     summary_url = df.loc[i,'summary_page_url']
#     print(summary_url)
    
#     r = requests.get(summary_url) #location in my dataframe
#     s = bs(r.text)
#     ex_all_tables = s.find_all('table')
    
#     for table in ex_all_tables:
#         all_rows = table.find_all('tr')
#         found_source = False
#         for row in all_rows:
#     #         print(row)
#             if ('Source' in row.text):
#                 found_source = True
#                 source_strg = row.find_all('a')[0].text
#                 print(source_strg)
#                 all_movies_source_list.append(source_strg)
#                 break
        
#         if (found_source):
#             break
#     if (not found_source):
#         source_strg = 'nf'
#         print()
#         all_movies_source_list.append(source_strg)
    

# source_2019_df = pd.DataFrame(all_movies_source_list, columns = ['Source'])        
# source_2019_df.to_pickle('source_2019_pk') 
    

https://www.the-numbers.com/movie/Avengers-Endgame-(2019)#tab=summary
Based on Comic/Graphic Novel
https://www.the-numbers.com/movie/Lion-King-The-(Live-Action)-(2019)#tab=summary
Remake
https://www.the-numbers.com/movie/Frozen-II-(2019)#tab=summary
Based on Folk Tale/Legend/Fairytale
https://www.the-numbers.com/movie/Toy-Story-4-(2019)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Captain-Marvel-(2019)#tab=summary
Based on Comic/Graphic Novel
https://www.the-numbers.com/movie/Star-Wars-The-Rise-of-Skywalker-(2019)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Spider-Man-Far-From-Home-(2019)#tab=summary
Based on Comic/Graphic Novel
https://www.the-numbers.com/movie/Aladdin-(2019)#tab=summary
Remake
https://www.the-numbers.com/movie/Joker-(2019)#tab=summary
Based on Comic/Graphic Novel
https://www.the-numbers.com/movie/It-Chapter-Two-(2019)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Jumanji-The-Next-Level#tab=summary
Origina

Original Screenplay
https://www.the-numbers.com/movie/Bombshell#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Judy-(2019)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/gi-saeng-chung-(South-Korea)-(2019)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Fighting-With-My-Family-(2019)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/On-the-Basis-of-Sex-(2018)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Dogs-Journey-A-(2019)#tab=summary
Based on Fiction Book/Short Story
https://www.the-numbers.com/movie/Booksmart-(2019)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Stuber-(2019)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/47-Meters-Down-Uncaged-(2019)#tab=summary
Based on Movie
https://www.the-numbers.com/movie/Black-and-Blue-(2019)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Hellboy-(2019)#tab=summary
Remak

Based on Movie
https://www.the-numbers.com/movie/Maiden-(Documentary)-(UK)-(2019)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Honey-Boy-(2019)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Friends-25th-The-One-With-the-Anniversary#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Yip-Man-4-(Hong-Kong)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Fantastic-Beasts-The-Crimes-of-Grindelwald-(2018)#tab=summary
Spin-Off
https://www.the-numbers.com/movie/Kalank-(India)-(2019)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Bharat-(India)-(2019)#tab=summary
Based on Movie
https://www.the-numbers.com/movie/Todos-lo-saben-(Spain)-(2018)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Todos-Caen-(Mexico)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Saaho-(India)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Instant-Family-(2

Based on Real Life Events
https://www.the-numbers.com/movie/Nancy-Drew-and-the-Hidden-Staircase-(2019)#tab=summary
Based on Fiction Book/Short Story
https://www.the-numbers.com/movie/Wizard-of-Oz-The#tab=summary
Based on Fiction Book/Short Story
https://www.the-numbers.com/movie/Miles-Davis-Birth-of-the-Cool#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Chaperone-The-(2019)#tab=summary
Based on Fiction Book/Short Story
https://www.the-numbers.com/movie/Roger-Waters-Us-Plus-Them#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Hai-Phuong-(Vietnam)-(2019)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Bethany-Hamilton-Unstoppable-(2019)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Public-The-(2019)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Sky-is-Pink-The-(India)-(2019)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Shining-The#tab=summary
Based 

Based on Real Life Events
https://www.the-numbers.com/movie/Family-(Film-Arcade)-(2019)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Mr-Klein-(France)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Synonymes-(France)-(2019)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Song-of-Names-The-(Canada)#tab=summary
Based on Fiction Book/Short Story
https://www.the-numbers.com/movie/To-Dust-(2019)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Robin-Hood-(2018)#tab=summary
Based on Folk Tale/Legend/Fairytale
https://www.the-numbers.com/movie/Ponokku-tanpen-gekijo-Chiisa-na-eiyu-Kani-to-tamago-to-tomei-ningen-(Japan)-(2018)#tab=summary
Compilation
https://www.the-numbers.com/movie/Warrior-Queen-of-Jhansi-The-(UK)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Creed-II-(2018)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Indias-Most-Wanted-(India)-(2019)#tab=summary
Based on

Original Screenplay
https://www.the-numbers.com/movie/Deadpool-2#tab=summary
Based on Comic/Graphic Novel
https://www.the-numbers.com/movie/Old-Man-and-the-Gun-The-(2018)#tab=summary
Based on Factual Book/Article
https://www.the-numbers.com/movie/Herederas-Las-(Paraguay)-(2018)#tab=summary

https://www.the-numbers.com/movie/homme-fidele-L-(France)-(2018)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Fast-Color-(2019)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Clemency-(2019)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Triple-Threat-(Thailand)-(2019)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Xue-Bao-(China)-(2019)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Nutcracker-and-the-Four-Realms-The-(2018)#tab=summary
Based on Folk Tale/Legend/Fairytale
https://www.the-numbers.com/movie/Hale-County-This-Morning-This-Evening-(Documentary)-(2018)#tab=summary
Based on Real Life Events
htt

Original Screenplay
https://www.the-numbers.com/movie/Leaving-Home-Coming-Home-A-Portrait-of-Robert-Frank-(UK)-(Documentary)-(2019)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Day-Shall-Come-The-(2019)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Wrestle-(Documentary)-(2019)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Donnybrook-(2019)#tab=summary
Based on Fiction Book/Short Story
https://www.the-numbers.com/movie/Seuwingkizeu-(South-Korea)-(2018)#tab=summary
Based on Musical or Opera
https://www.the-numbers.com/movie/Retour-du-Heros-Le-(France)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Chasing-Einstein#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/religieuse-La-(France)-(1966)#tab=summary
Based on Fiction Book/Short Story
https://www.the-numbers.com/movie/Beautiful-Boy-(2018)#tab=summary
Based on Factual Book/Article
https://www.the-numbers.com/movie/paranza-d

Based on Real Life Events
https://www.the-numbers.com/movie/Tyrel-(2018)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Aspern-Papers-The-(UK)-(2019)#tab=summary
Based on Fiction Book/Short Story
https://www.the-numbers.com/movie/Liyana-(Swaziland)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Cracked-Up#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Advocate-(Switzerland)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Frantz-Fanon-Black-Skin-White-Mask-(UK)-(1995)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Pulipdeul-(South-Korea)(2018)#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/That-Part-Feeling-The-Universe-of-Arvo-Part-(Netherlands)-(Documentary)-(2019)#tab=summary
Based on Real Life Events
https://www.the-numbers.com/movie/Dead-Center-The#tab=summary
Original Screenplay
https://www.the-numbers.com/movie/Ce-magnifique-gateau-(Belgium)-(20

In [84]:
# len(all_movies_source_list)
# source_2019_df

In [85]:
# source_2019_df[source_2019_df['Source'] == 'nf']

In [112]:
# all_movies_creative_type_list = []

# for i in range(row_count):
#     summary_url = df.loc[i,'summary_page_url']
    
#     r = requests.get(summary_url) #location in my dataframe
#     s = bs(r.text)
#     ex_all_tables = s.find_all('table')
    
#     for table in ex_all_tables:
#         all_rows = table.find_all('tr')
#         found_ctype = False
#         for row in all_rows:
# #             print(row.text)
#             if ('Creative' in row.text and 'Type' in row.text):
#                 found_ctype = True
# #                 print(row)
#                 ctype = row.find_all('a')[0].text
#                 print(ctype)
                
#                 all_movies_creative_type_list.append(ctype)
#                 break
        
#         if (found_ctype):
#             break
#     if (not found_ctype):
#         ctype = 'nf'
#         print('df')
#         all_movies_creative_type_list.append(ctype)
    

# creative_type_2019_df = pd.DataFrame(all_movies_creative_type_list,
#                                      columns = ['Creative_Type'])        
# creative_type_2019_df.to_pickle('creative_type_2019_pk') 

Super Hero
Kids Fiction
Kids Fiction
Kids Fiction
Super Hero
Science Fiction
Super Hero
Fantasy
Contemporary Fiction
Contemporary Fiction
Science Fiction
Contemporary Fiction
Contemporary Fiction
Contemporary Fiction
Kids Fiction
Kids Fiction
Kids Fiction
Dramatization
Super Hero
Super Hero
Contemporary Fiction
Kids Fiction
Fantasy
Contemporary Fiction
Science Fiction
Dramatization
Dramatization
Kids Fiction
Dramatization
Kids Fiction
Historical Fiction
Dramatization
Science Fiction
Contemporary Fiction
Science Fiction
Contemporary Fiction
Contemporary Fiction
Contemporary Fiction
Contemporary Fiction
Historical Fiction
Contemporary Fiction
Historical Fiction
Super Hero
Science Fiction
Kids Fiction
Kids Fiction
Dramatization
Dramatization
Super Hero
Science Fiction
Contemporary Fiction
Contemporary Fiction
Contemporary Fiction
Dramatization
Science Fiction
Contemporary Fiction
Science Fiction
Kids Fiction
Contemporary Fiction
Contemporary Fiction
Kids Fiction
Contemporary Fiction
Conte

Historical Fiction
Factual
Contemporary Fiction
Historical Fiction
Contemporary Fiction
Contemporary Fiction
Contemporary Fiction
Factual
Contemporary Fiction
Factual
Contemporary Fiction
Historical Fiction
Historical Fiction
Factual
Historical Fiction
Contemporary Fiction
Contemporary Fiction
Factual
Factual
Contemporary Fiction
Science Fiction
Contemporary Fiction
Historical Fiction
Contemporary Fiction
Contemporary Fiction
Contemporary Fiction
Contemporary Fiction
Factual
Factual
Factual
Contemporary Fiction
Contemporary Fiction
Factual
Contemporary Fiction
Contemporary Fiction
Factual
Factual
Contemporary Fiction
Contemporary Fiction
Factual
Historical Fiction
Factual
Dramatization
Contemporary Fiction
Dramatization
Factual
Contemporary Fiction
Contemporary Fiction
Contemporary Fiction
Contemporary Fiction
Science Fiction
Historical Fiction
Factual
Factual
Dramatization
Contemporary Fiction
Contemporary Fiction
Contemporary Fiction
Factual
Factual
Factual
Factual
Factual
Contempora

In [113]:
len(all_movies_creative_type_list)
creative_type_2019_df

,Creative_Type
0,Super Hero
1,Kids Fiction
2,Kids Fiction
3,Kids Fiction
4,Super Hero
...,...
675,Factual
676,Contemporary Fiction
677,Factual
678,Historical Fiction


In [114]:
creative_type_2019_df[creative_type_2019_df['Creative_Type'] == 'nf']

,Creative_Type
168,nf
174,nf
304,nf
325,nf
348,nf
379,nf
388,nf
427,nf
496,nf
511,nf
